## MDLE - Project 2

In [63]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ReadCSV").getOrCreate()
#tracks = spark.read.csv('fma_metadata/tracks.csv', header='True')

tracks = spark.read.format("csv") \
    .option("header", "true") \
    .load("fma_metadata/tracks.csv")

tracks.show()



'''small_tracks = tracks.where((tracks.subset == 'small'))
small_tracks.show()
print(small_tracks.select('set31'))'''

23/04/17 20:00:17 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , album, album, album, album, album, album, album, album, album, album, album, album, album, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, set, set, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track
 Schema: _c0, album1, album2, album3, album4, album5, album6, album7, album8, album9, album10, album11, album12, album13, artist14, artist15, artist16, artist17, artist18, artist19, artist20, artist21, artist22, artist23, artist24, artist25, artist26, artist27, artist28, artist29, artist30, set31, set32, track33, track34, track35, track36, track37, track38, track39, track40, track41, track42, track43, track44, track45, track46, track47, track48, track49, track50, track51, track52
Expected: _c0 but found: 
CSV file: file:///

"small_tracks = tracks.where((tracks.subset == 'small'))\nsmall_tracks.show()\nprint(small_tracks.select('set31'))"

In [64]:
from pyspark.sql.functions import col


first_two_rows = tracks.take(2)
first_row = first_two_rows[0]
second_row = first_two_rows[1]
#new_column_names = [str(cell) for cell in first_row]
new_column_names = [str(cell) for cell in first_row[1:]] + [str(second_row[0])]
renamed_tracks = tracks.toDF(*new_column_names)





23/04/17 20:00:17 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , album, album, album, album, album, album, album, album, album, album, album, album, album, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, set, set, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track
 Schema: _c0, album1, album2, album3, album4, album5, album6, album7, album8, album9, album10, album11, album12, album13, artist14, artist15, artist16, artist17, artist18, artist19, artist20, artist21, artist22, artist23, artist24, artist25, artist26, artist27, artist28, artist29, artist30, set31, set32, track33, track34, track35, track36, track37, track38, track39, track40, track41, track42, track43, track44, track45, track46, track47, track48, track49, track50, track51, track52
Expected: _c0 but found: 
CSV file: file:///

In [65]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, monotonically_increasing_id
data_with_id = renamed_tracks.withColumn("unique_id", monotonically_increasing_id())
min_id = data_with_id.select("unique_id").orderBy("unique_id").first()[0]
second_id = data_with_id.select("unique_id").orderBy("unique_id").take(2)[-1][0]

data_without_first_row = data_with_id.filter(col("unique_id") != min_id).drop("unique_id")
data_without_second_row = data_without_first_row.filter(col("unique_id") != second_id).drop("unique_id")

data_without_second_row.show()


23/04/17 20:00:19 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , album, album, album, album, album, album, album, album, album, album, album, album, album, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, set, set, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track
 Schema: _c0, album1, album2, album3, album4, album5, album6, album7, album8, album9, album10, album11, album12, album13, artist14, artist15, artist16, artist17, artist18, artist19, artist20, artist21, artist22, artist23, artist24, artist25, artist26, artist27, artist28, artist29, artist30, set31, set32, track33, track34, track35, track36, track37, track38, track39, track40, track41, track42, track43, track44, track45, track46, track47, track48, track49, track50, track51, track52
Expected: _c0 but found: 
CSV file: file:///

In [66]:
col_list = data_without_second_row.columns
shifted_col_list = [col_list[-1]] + col_list[:-1]
shifted_df = data_without_second_row.toDF(*shifted_col_list)
shifted_df.show()

23/04/17 20:00:19 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , album, album, album, album, album, album, album, album, album, album, album, album, album, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, set, set, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track
 Schema: _c0, album1, album2, album3, album4, album5, album6, album7, album8, album9, album10, album11, album12, album13, artist14, artist15, artist16, artist17, artist18, artist19, artist20, artist21, artist22, artist23, artist24, artist25, artist26, artist27, artist28, artist29, artist30, set31, set32, track33, track34, track35, track36, track37, track38, track39, track40, track41, track42, track43, track44, track45, track46, track47, track48, track49, track50, track51, track52
Expected: _c0 but found: 
CSV file: file:///

In [73]:
shifted_df.where(shifted_df.subset == 'small').show()

23/04/17 20:17:32 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , album, album, album, album, album, album, album, album, album, album, album, album, album, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, artist, set, set, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track, track
 Schema: _c0, album1, album2, album3, album4, album5, album6, album7, album8, album9, album10, album11, album12, album13, artist14, artist15, artist16, artist17, artist18, artist19, artist20, artist21, artist22, artist23, artist24, artist25, artist26, artist27, artist28, artist29, artist30, set31, set32, track33, track34, track35, track36, track37, track38, track39, track40, track41, track42, track43, track44, track45, track46, track47, track48, track49, track50, track51, track52
Expected: _c0 but found: 
CSV file: file:///

In [80]:
from pyspark.sql.functions import col, when
def merge_columns(df, col_name):
    duplicate_columns = [c for c in df.columns if c == col_name]
    
    if len(duplicate_columns) <= 1:
        return df
    
    merged_col = col(duplicate_columns[0])
    
    for c in duplicate_columns[1:]:
        merged_col = when(merged_col == 0, col(c)).otherwise(merged_col)
    
    df = df.drop(*duplicate_columns).withColumn(col_name, merged_col)
    return df

In [81]:
merged_comments = merge_columns(shifted_df, 'comments')

AnalysisException: Column 'comments' does not exist. Did you mean one of the following? [composer, listens, listens, members, genres, license, name, number, subset, bit_rate, favorites, favorites, favorites, genre_top, interest, location, split, tags, tags, tags, title, title, tracks, type, website, bio, duration, engineer, id, id, latitude, longitude, lyricist, producer, track_id, genres_all, information, information, publisher, date_created, date_created, date_created, date_released, date_recorded, language_code, active_year_end, related_projects, wikipedia_page, associated_labels, active_year_begin];
'Project [track_id#15756, date_created#15758, date_released#15759, engineer#15760, favorites#15761, id#15762, information#15763, listens#15764, producer#15765, tags#15766, title#15767, tracks#15768, type#15769, active_year_begin#15770, active_year_end#15771, associated_labels#15772, bio#15773, date_created#15775, favorites#15776, id#15777, latitude#15778, location#15779, longitude#15780